In [ ]:
import argparse
import os
import torch
from ACBiSeNetV6 import ACBiSeNetV6

parser = argparse.ArgumentParser(description='ACB Conversion')
parser.add_argument('load', metavar='LOAD', help='path to the weights file')
parser.add_argument('save', metavar='SAVE', help='path to the weights file')
parser.add_argument('-model', '--arch', metavar='ARCH', default='ACBiSeNetV2')

def convert():
    args = parser.parse_args()

    train_model = ACBiSeNetV6(6,'mobilenetv3')

    if os.path.isfile(args.load):
        print(f"=> loading checkpoint '{args.load}'")
        checkpoint = torch.load(args.load)
        
        if 'state_dict' in checkpoint:
            checkpoint = checkpoint['state_dict']
        ckpt = {k.replace('module.', ''): v for k, v in checkpoint.items()}  # strip the names
        train_model.load_state_dict(ckpt)
    else:
        print("=> no checkpoint found at '{args.load}'")

    for m in train_model.modules():
        if hasattr(m, 'switch_to_deploy'):
            m.switch_to_deploy()
    torch.save(train_model.state_dict(), args.save)

In [ ]:
if __name__ == '__main__':
    train_model = ACBiSeNetV2(6,'mobilenetv3')
    train_model = nn.DataParallel(train_model);   #多gpu训练的模型，要加这一句才行
    train_model.cuda();
    
    checkpoint = torch.load('/root/master/ATL/code/code/checkpoint/Potsdam/acbisenetv2/05-14-02_10_10/model_weights.pth')
    save_path = '/root/master/ATL/code/code/checkpoint/Potsdam/acbisenetv2/05-14-02_10_10/model_weights_deploy.pth'
    
    if 'state_dict' in checkpoint:
        checkpoint = checkpoint['state_dict']
    # ckpt = {k.replace('module.', ''): v for k, v in checkpoint.items()}  # strip the names #把多dpu训练的模型给那啥了
    train_model.load_state_dict(checkpoint) #ckpt

    for m in train_model.modules():
        if hasattr(m, 'switch_to_deploy'):
            m.switch_to_deploy()
        torch.save(train_model.state_dict(), save_path)